In [1]:
#Dependencies
import deepmusic.moduleloader import ModuleLoader
#predicts next key
from deepmusic.keyboardcell import KeyboardCell
#encapsulate song data so we can run get_scale, get_relative_methods
import deepmusic.songstruct as music
import numpy as np #for random numbers
import tensorflow as tf

ModuleNotFoundError: No module named 'deepmusic'

In [ ]:
def build_network(self):
    #crete computation graph, encapsulates session and graph initialization
    input_dim = ModuleLoader.batch_builders.get_module().get_input_dim()
    
    #note data
    with tf.name('placeholder_inputs'):
        self.inputs = [
            tf.placeholder(
            tf.float32, #numerical data
            [self.args.batch_size, input_dim], #how much data
            name = 'input'
            )
        ]
    #targets 88 key, binary classification problem
    with tf.name_scope('placeholder_targets'):
        self.targets = [
            tf.placeholder(tf.int32, #0 or 1
                          [self.batch_size],
                          name = 'target') 
        ]
    #hidden state, dont konw what hidden state means/
    with tf.name_scope ('placeholder_use_prev'):
        self.use_prev = [
            tf.placeholder(
            tf.bool, 
            [],
            name = 'use_prev')
        ]
    
    #define our network
    self.loop_processing = ModuleLoader.loop_processings.build_module(self.args)
    def loop_rnn(prev, i):
        next_input = self.loop_processing(prev)
        return tf.cond(self.prev(i), lambda: next_input, lambda:self.inputs[i])
    
    #build seq2seq model
    self.outputs, self.final_state = tf.nn.seq2seq.rnn_decoder(
        decoder_inputs = self.inputs,
        initial_state = None, 
        cell = KeyboardCell, 
        loop_function = loop_rnn)#because definedin keyboard cell
    
    #training step
    #defining the loss function
    loss_fct = tf.nn.seq2seq.sequence_loss(
    self.outputs,
    self.targets, 
    softmax_loss_function = tf.nn.softmax.cross_entropy_with_logits,
    average_across_timesteps = True,
    average_across_batch = True,
    )
    
    #initialize the optimizer, minimize the loss
    opt = tf.train.AdamOptimizer(
        learning_rate = self.current_learning_rate,
        beta1 = .9,
        beta2 = .999,
        epsilon = 1e-08)
    
    self.opt_op = opt.minimize(loss_fct)